# Documenting how to compute specificity for RNAseq datasets
- In this document I want to explain step by step how specificity is computed. 
- Because these codes are not yet generalizable, meaning that there are a lot of hard-coded path. Therefore, I would suggest:
    - first, go over this notebook to understand how these statistics are
    - second, copy over relevant codes to a fresh python script so that you can scale it up easily

## Specificity
- There are 3 ways (that I am aware of) to compute specificity
    - Script from Rachel (original script is `single_cell_MAGMA_files_v2.py` but I have modified this to `compute_sumstat_magma.py`). For convenience, I will call this method `spec_vanilla`
    - EWCE: method used in Skene papers. I will call this `spec_ewce`
    - CELLEX: method from Timshel et al. I will call this `spec_cellex`
- Below I will attempt to demonstrate how to calculate these

### spec_vanilla
- TL;DR: 
    - In this method, for each gene, the specifity value is computed for each cell type. Thus, for each gene, the sum of the specificity values should sum up to 1
    - First, for each gene, mean expression is computed for each cell type. Here, gene expression value has already been converted to per million. Then, the specificity is calculated by dividing the mean for each cell type by the sum of mean expression across all cell types
    - In this method, for each gene, the mean expression (per million) for each cell type is 0 if it's less than 1. The rationale is that if the mean expression for that cell type is less than 1 (per million), then it is lowly expressed and should be removed. 
        - However, this is somehow assumed that the gene where the mean expression for a specific cell type is low, it means that it is "bad" and should be removed. But what if it is actually lowly expressed? 
    - Then, after the specificity for each gene for each cell type is done, to get the top X% specificity
        - For each cell type, remove genes where specificity is 0, then get the top X% (this is done in Rachel's script)
            - Consequence: the number of top X% expressed genes per cell type is going to differ between cell types. This is because the number of genes that are removed due to specificity being 0 is different per cell type.

- In the following code I will walk through an example.

In [1]:
import scanpy as sc
import anndata
import pandas as pd

- First, read in and clean the h5ad file

In [2]:
adata_path = "scrna/41_Siletti_Cerebellum.CBV_Human_2022/41_Siletti_Cerebellum.CBV_Human_2022.h5ad" #this is the h5ad file after preprocessing
adata_original = anndata.read(adata_path)
print(f"Originally, adata has {adata_original.shape[0]} rows (cells) and {adata_original.shape[1]} columns (genes)")

# because the scRNAseq data contains genes that don't have ensemble, it is cleaner to create a filtered adata where:
# genes that were not convertible to ensembl
adata_original.var_names_make_unique()
gene_list = adata_original.var[adata_original.var["ensembl"].notnull()].index.values.tolist() #this obtains a list of gene in symbol that has an ensemble conversion
adata = adata_original[:, gene_list].copy() #subset based on the genes (keep the genes if it was converted to ensemble successfully)

# now, adata.X is the matrix in numpy format where row is cell ID and column is the gene\
print(f"After cleaning, adata has {adata.shape[0]} rows (cells) and {adata.shape[1]} columns (genes)")

c:\Users\tph205\AppData\Local\anaconda3\envs\jupyter_env\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Originally, adata has 71852 rows (cells) and 36515 columns (genes)
After cleaning, adata has 71852 rows (cells) and 24617 columns (genes)


In [3]:
# adata.obs.columns
adata.obs["supercluster_term"].unique()

['Committed oligodendrocyte precursor', 'Oligodendrocyte', 'Oligodendrocyte precursor', 'Splatter', 'Upper rhombic lip', ..., 'Bergmann glia', 'Ependymal', 'Vascular', 'Fibroblast', 'Microglia']
Length: 13
Categories (13, object): ['Astrocyte', 'Bergmann glia', 'Cerebellar inhibitory', 'Committed oligodendrocyte precursor', ..., 'Oligodendrocyte precursor', 'Splatter', 'Upper rhombic lip', 'Vascular']

- Second, compute specificity

In [4]:
ct_colname = "supercluster_term"
# first normalize
# normalize counts per cell (in place)
sc.pp.normalize_total(adata, target_sum=1e6)

# define the genes in the dataset
genes = adata.var["ensembl"].to_list()

# define the available cell types in the dataset
cts = adata.obs[ct_colname].dropna().unique()

# before log-transforming, we determine a filtering on a minimum of 1 transcript per million per celltype
# start with computing the mean cpM per cell type
means_cell_counts_pM = pd.DataFrame(data=None, index=cts, columns=genes, dtype=float)  # rows is the cell type; columns is the gene
for ct in cts:
    means_cell_counts_pM.loc[ct, :] = adata[adata.obs[ct_colname] == ct, :].X.mean(0)

print(f"The mean dataframe has {means_cell_counts_pM.shape[0]} rows (cell types) and {means_cell_counts_pM.shape[1]} columns (genes)")
print("View the dataframe means_cell_counts_pM")
print(means_cell_counts_pM.iloc[:, : 5])


# the low_filter matrix contains zeros for genes that have < 1 count pM in a certain cell type
# this matrix will be used to multiply the same data when it is log-transformed
low_filter = (1 - (means_cell_counts_pM < 1))  # 0 is <1, 1 if >1
print(f"The low filter dataframe has {low_filter.shape[0]} rows (cell types) and {low_filter.shape[1]} columns (genes)")
print("View the dataframe low filter")
print(low_filter.iloc[:, : 5])

# specificity values
# set NaNs (i.e. no expression in any of the cells/celltypes) in the mean matrix per cell type to zero
spec_cell_counts_pM = ((means_cell_counts_pM.mul(low_filter)) / (means_cell_counts_pM.mul(low_filter)).sum(axis=0)).fillna(0)
# more explanation: first, for each gene, if the mean expression for a cell type is less than 1, it will be first converted to 0. Then, specificity is computed by dividing mean expression for a cell type by the sum mean expression across all cell types. In the case of NA due to dividing by 0, convert to 0 afterwards. 
print(f"The specificity dataframe has {spec_cell_counts_pM.shape[0]} rows (cell types) and {spec_cell_counts_pM.shape[1]} columns (genes)")
print("View the dataframe specificity")
print(spec_cell_counts_pM.iloc[:, : 5])

The mean dataframe has 13 rows (cell types) and 24617 columns (genes)
View the dataframe means_cell_counts_pM
                                     ENSG00000284678  ENSG00000167995   
Committed oligodendrocyte precursor         5.413892        88.919067  \
Oligodendrocyte                             0.900931        52.096020   
Oligodendrocyte precursor                   0.417204        19.098175   
Splatter                                    0.000000         0.000000   
Upper rhombic lip                           0.198289         1.025848   
Cerebellar inhibitory                       0.312472         0.523617   
Miscellaneous                               0.000000         0.460915   
Astrocyte                                   0.000000        26.481335   
Bergmann glia                               0.000000         2.707252   
Ependymal                                   0.000000         0.000000   
Vascular                                    0.000000         3.799043   
Fibroblast    

- Sanity check: examining the dataframe for specifity, we can confirm that for each gene, the sum of specificity across all cell types is summed up to 1. 

- Now onto how to obtain: for each gene, the top 10 percent genes that are specific for this cell type? And how to define the background genes to be used for LDSC-SEG? 
- What Rachel implemented originally in the script `single_cell_MAGMA_files_v2.py` is: 
    - for each cell type, remove the genes where specificity is 0.
    - for each cell type, obtain the top 10% of genes that are specifiic for that cell type using a list of genes where specificity value of 0 have been removed
    - What this means is that the number of top 10% specific genes for each cell type is going to differ (this is because the number of genes where specificity value of 0 is going to differ between cell types)
    - Then, **how to obtain a list of background genes?** The background genes are defined as the genes where the sum of specificity across all cell type is greater than 0

In [5]:
for ct in spec_cell_counts_pM.index:
    print(ct)
    sorted = spec_cell_counts_pM.loc[ct].sort_values(ascending=False)
    sorted=sorted[sorted>0]
    numexpressed = len(sorted)
    maxincl=sorted[round(0.1*numexpressed)-1]
    new = spec_cell_counts_pM.columns[spec_cell_counts_pM.loc[ct] >= maxincl]
    print(new)

Committed oligodendrocyte precursor
Index(['ENSG00000284678', 'ENSG00000167995', 'ENSG00000204655',
       'ENSG00000253807', 'ENSG00000071991', 'ENSG00000160951',
       'ENSG00000158859', 'ENSG00000167123', 'ENSG00000130158',
       'ENSG00000167565',
       ...
       'ENSG00000181031', 'ENSG00000141582', 'ENSG00000117009',
       'ENSG00000093072', 'ENSG00000261485', 'ENSG00000156697',
       'ENSG00000259642', 'ENSG00000122025', 'ENSG00000160570',
       'ENSG00000177548'],
      dtype='object', length=1261)
Oligodendrocyte
Index(['ENSG00000167995', 'ENSG00000204655', 'ENSG00000253807',
       'ENSG00000169247', 'ENSG00000071991', 'ENSG00000167641',
       'ENSG00000158859', 'ENSG00000167123', 'ENSG00000168314',
       'ENSG00000105695',
       ...
       'ENSG00000086288', 'ENSG00000232684', 'ENSG00000105643',
       'ENSG00000180539', 'ENSG00000076258', 'ENSG00000162729',
       'ENSG00000236463', 'ENSG00000158406', 'ENSG00000213440',
       'ENSG00000180739'],
      dtype='obje

- As seen from above, the number of genes that are specific to each cell type is different between cell types. 
  
**Overall thoughts**
- This is probably ok to compute it this way for now and we will make sure to explain what we did

- Now print out the top 10% specific genes to the output file

In [31]:
# Define a function to obtain the top X percent genes that are specific for a cell type
import os
def get_top_perc(ctmatrix,perc, ldsc_outdir, magma_outdir, method):
        """ ctmatrix is a pandas dataframe containing a parameter of interest; rows (celltypes) and columns (genes)
            the top "perc" percentage of genes for each celltype will be written to the outputfile 
        """ 
        # Initialize outfile for magma
        magma_outfile = open(os.path.join(magma_outdir, "spec_" + str(method) + "_top10.txt"), "w")
        for ct in ctmatrix.index:
            sorted = ctmatrix.loc[ct].sort_values(ascending=False)
            # remove the nonzero values
            sorted=sorted[sorted>0]
            numexpressed = len(sorted)
            maxincl=sorted[round(perc*numexpressed)-1]
            ct_fmt = ct.replace(' ',"_")
            print(ct)
            print(ct_fmt)
            # write outfile for ldsc
            ct_outfile = open(os.path.join(ldsc_outdir, ct_fmt + ".txt"), "w")
            ct_outfile.write('\n'.join(ctmatrix.columns[ctmatrix.loc[ct] >= maxincl]))
            ct_outfile.close()
            # write outfile for magma
            magma_out = [ct_fmt]
            for i in ctmatrix.columns[ctmatrix.loc[ct] >= maxincl]:
                magma_out.append(i)
            print("\t".join(magma_out), file=magma_outfile)
        
        # add a line with all the genes that are expressed in this dataset:
        all_outfile = open(os.path.join(ldsc_outdir, "All.txt"), "w")
        all_outfile.write('\n'.join(ctmatrix.columns[ctmatrix.sum(axis=0) > 0]))
        all_outfile.close()

In [32]:
get_top_perc(spec_cell_counts_pM, 0.1, "ldsc/spec_vanilla/41_Siletti_Cerebellum.CBV_Human_2022/genelists", "magma/spec_vanilla_top10/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar", "vanilla")

Committed oligodendrocyte precursor
Committed_oligodendrocyte_precursor
Oligodendrocyte
Oligodendrocyte
Oligodendrocyte precursor
Oligodendrocyte_precursor
Splatter
Splatter
Upper rhombic lip
Upper_rhombic_lip
Cerebellar inhibitory
Cerebellar_inhibitory
Miscellaneous
Miscellaneous
Astrocyte
Astrocyte
Bergmann glia
Bergmann_glia
Ependymal
Ependymal
Vascular
Vascular
Fibroblast
Fibroblast
Microglia
Microglia


# save for running magma linear
- Because the file format for ldsc is different from magma, I need to save it differently to be compatible to use with magma

In [33]:
spec_cell_counts_pM.index = [w.replace(' ', '_') for w in spec_cell_counts_pM.index.values]
spec_cell_counts_pM_t = spec_cell_counts_pM.T
spec_cell_counts_pM_t.index.name = "GENE"
spec_cell_counts_pM_t.reset_index(inplace=True)
spec_cell_counts_pM_out = spec_cell_counts_pM_t.drop_duplicates(subset=["GENE"])
spec_cell_counts_pM_out.to_csv(os.path.join("magma/spec_vanilla_linear/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar/spec_vanilla_linear.txt"), sep="\t", index=False) # save to a file)

### spec_ewce
- Skene et al. 2018 (Reference (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6477180/): `It was calculated using the generate.celltype.data function of the EWCE package` in the method section) computed specificity using an R package called EWCE
- Step 1: run EWCE. 
    - Use the Rscript `process_scrna_ewce.R`. Example usage in snakemake (this is just an example and one needs to adapt accordingly):
    
    ```
    rule run_ewce:
        input:
            h5ad = os.path.join(config["processed_h5ad_dir"], "{h5ad_id}_processed_magma_celltyping.h5ad")
        output:
            os.path.join(config["processed_h5ad_dir"], "{h5ad_id}_" + "siletti.rda")
        params:
            outdir = config["processed_h5ad_dir"],
            file_prefix = "{h5ad_id}",
            groupName = "siletti"
        shell:
            """
            Rscript process_scrna_ewce.R --h5ad {input.h5ad} --outdir {params.outdir} --file_prefix {params.file_prefix} --groupName {params.groupName}
            """
    ```
- Step 2: Convert to text file
    - Use the Rscript `process_ewce.R` (currently hard coding for 1 example, #TODO: generalize this script)
    - Example output of this script for specificity
    
    ```
    GENE    Astrocyte       Bergmann_glia   Cerebellar_inhibitory   Committed_oligodendrocyte_precursor     Ependymal     Fibroblast      Microglia       Miscellaneous   Oligodendrocyte Oligodendrocyte_precursor       Splatter      Upper_rhombic_lip       Vascular
    ENSG00000121410 0.0193375341490014      0.0178245768825862      0.0261292653835096      0       0       0.0840939042241185    0.0959529061791269      0       0.395047887373253       0.0103135545556687      0       0.14467409748486      0.206626258236432
    ```
    - Note that when implementing ewce, it computes both the specificity and mean. In addition, in the `process_ewce.R` script, I'm also computing the average as well. Here is an example:
    ```
    GENE    Astrocyte       Bergmann_glia   Cerebellar_inhibitory   Committed_oligodendrocyte_precursor     Ependymal     Fibroblast      Microglia       Miscellaneous   Oligodendrocyte Oligodendrocyte_precursor       Splatter      Upper_rhombic_lip       Vascular        Average
    ENSG00000121410 0.00492610837438424     0.00619990926962044     0.0207334861429464      0       0       0.0173410404624277    0.0153738644304682      0       0.122331879733762       0.00362506473329881     0       0.0437523585429199    0.0411255411255411      0.0211853271396437
    ENSG00000148584 0       0.000453651897777106    0.0738695561575328      0       0       0       0       0.0857142857142857    0.00160661005278862     0.00207146556188503     0       0.000843525938422606    0       0.0126583919478994
    ENSG00000175899 0.00985221674876847     0.0562528353243611      0.0365701187747447      0.0333333333333333
    0     0.0346820809248555      2.14325646401118        0.242857142857143       0.0406242827633693      0.0515277058518902    1       0.0429754267575307      4.33116883116883        0.617161572193539
    ``` 
- Step 3: Obtain the genes with top X% in specificity
    - Note: Here I would like to use the same Python function as above (function `get_top_perc`). However, in theory, if you don't want to have to run multiple script, you can update the R script above (`process_ewce.R`) to get the top X% instead
    

In [14]:
# Example python code to obtain the genes with top X% in specificity
# load in the file
spec_ewce_orig = pd.read_csv("scrna/41_Siletti_Cerebellum.CBV_Human_2022/spec_ewce_linear.tsv", sep="\t")
spec_ewce_orig.set_index('GENE', inplace=True)
print(spec_ewce_orig.head)
spec_ewce = spec_ewce_orig.transpose()
print(spec_ewce.head)


<bound method NDFrame.head of                  Astrocyte  Bergmann_glia  Cerebellar_inhibitory   
GENE                                                               
ENSG00000121410   0.019338       0.017825               0.026129  \
ENSG00000148584   0.000000       0.005407               0.385931   
ENSG00000175899   0.000962       0.004023               0.001146   
ENSG00000166535   0.480268       0.072990               0.068351   
ENSG00000256069   0.017136       0.078206               0.025598   
...                    ...            ...                    ...   
ENSG00000203995   0.000000       0.056879               0.177211   
ENSG00000162378   0.059627       0.076378               0.160735   
ENSG00000159840   0.071161       0.033241               0.076343   
ENSG00000074755   0.104901       0.073885               0.091331   
ENSG00000036549   0.043547       0.061985               0.041524   

                 Committed_oligodendrocyte_precursor  Ependymal  Fibroblast   
GENE  

In [15]:
#Sanity check to make sure that the sum of the columns is 1
spec_ewce['ENSG00000175899'].sum()
spec_ewce['ENSG00000198455'].sum()

get_top_perc(spec_ewce, 0.1, "ldsc/spec_ewce/41_Siletti_Cerebellum.CBV_Human_2022/genelists", "magma/spec_ewce_top10/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar")

- For magma linear, copy over the file: 
```
cp ../scrna/41_Siletti_Cerebellum.CBV_Human_2022/spec_ewce_linear.tsv spec_ewce_linear/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar/
```

### spec_cellex
- cellex is a method developed and used in Timshel et al. 2020 (https://github.com/perslab/CELLEX)
- Step 1: run cellex
    - Note #1: because each scRNASeq data needs to be tailored, I suggest that the script used to compute specificity using cellex should also needs to be tailored and adapted
    - Example: `process_scrna_cellex_siletti.py` (location on snellius home directory: `/gpfs/home6/tphung/projects/alc_jeanne/code/ct_enrichment/cellect/cellex/process_scrna_cellex_siletti.py`)
    - Note #2: I have computed specificity using cellex for a few scRNAseq data already. They are stored in the shared directory on snellius: `/gpfs/work5/0/vusr0480/Processed_scRNA/data/cellex/`
    - Example output: 
    ```
    head 41_Siletti_Cerebellum.CBV_Human_2022.esmu_fmt.csv
    gene,Astrocyte,Bergmann_glia,Cerebellar_inhibitory,Committed_oligodendrocyte_precursor,Ependymal,Fibroblast,Microglia,Miscellaneous,Oligodendrocyte,Oligodendrocyte_precursor,Splatter,Upper_rhombic_lip,Vascular
    ENSG00000284678,0.0,0.0,0.0455216731859184,0.6023177951786117,0.0,0.0,0.3862554246926514,0.0,0.1257935750541277,0.0405809483126868,0.0,0.0,0.0
    ENSG00000167995,0.2786127578454396,0.0,0.0,0.8486931953358344,0.0,0.0,0.1147825655528267,0.0,0.7639259838721524,0.3911041191936009,0.0,0.0,0.0
    ENSG00000204655,0.0,0.0,0.0,0.7199639397931027,0.0,0.0,0.0,0.0,0.9601996237244824,0.0,0.0,0.0,0.0
    ENSG00000253807,0.0579020801623541,0.0,0.0,0.9452305902508408,0.0,0.0,0.0,0.0,0.8377288286595166,0.4817267013403095,0.0,0.0,0.0
    ENSG00000169247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9772780005252236,0.0,0.0,0.0,0.0
    ENSG00000071991,0.0,0.0,0.0,0.6643209030420585,0.0,0.0,0.0,0.0,0.9677135629703932,0.0,0.0,0.0,0.0
    ENSG00000238057,0.0,0.2839402620494266,0.0,0.2051677573286847,0.0,0.0,0.8302622848540593,0.0,0.3562490855241139,0.0,0.0,0.0,0.0
    ENSG00000160951,0.0,0.0,0.0062664644368164,0.8592925359251826,0.0,0.0,0.1099966825726586,0.0,0.1692315950828765,0.0,0.0,0.0,0.2189957185631276
    ENSG00000167641,0.0,0.0,0.0,0.2637550975168584,0.0,0.0,0.0,0.0,0.8828416228884106,0.0,0.1872113356181184,0.0,0.2979612427739169
    ``` 
- Step 2: Obtain the genes with top X% in specificity
    - Note: Similar to the case of `cellex_ewce` above, Here I would like to use the same Python function as above (function `get_top_perc`).  

In [23]:
# Example python code to obtain the genes with top X% in specificity
# load in the file
spec_cellex_orig = pd.read_csv("scrna/41_Siletti_Cerebellum.CBV_Human_2022/41_Siletti_Cerebellum.CBV_Human_2022.esmu_fmt.csv")
spec_cellex_orig.set_index('gene', inplace=True)
print(spec_cellex_orig.head)
spec_cellex = spec_cellex_orig.transpose()
print(spec_cellex.head)

<bound method NDFrame.head of                  Astrocyte  Bergmann_glia  Cerebellar_inhibitory   
gene                                                               
ENSG00000284678   0.000000       0.000000               0.045522  \
ENSG00000167995   0.278613       0.000000               0.000000   
ENSG00000204655   0.000000       0.000000               0.000000   
ENSG00000253807   0.057902       0.000000               0.000000   
ENSG00000169247   0.000000       0.000000               0.000000   
...                    ...            ...                    ...   
ENSG00000167633   0.000000       0.000000               0.000000   
ENSG00000221957   0.000000       0.000000               0.000000   
ENSG00000005844   0.000000       0.000000               0.000000   
ENSG00000122122   0.000000       0.000000               0.000000   
ENSG00000246526   0.029521       0.060136               0.000000   

                 Committed_oligodendrocyte_precursor  Ependymal  Fibroblast   
gene  

In [35]:
#Sanity check to make sure that the sum of the columns is 1
print(spec_cellex['ENSG00000221957'].sum())
print(spec_cellex['ENSG00000005844'].sum())
print(spec_cellex['ENSG00000167633'].sum())
print(spec_cellex['ENSG00000204525'].sum())



0.9998340082613578
1.6080424367931252
0.9329457677807221
2.6925855842211823


- This is very strange. I'm not sure why the specificity per gene does not add up to 1. #TODO: INVESTIGATE CELLEX

In [17]:

get_top_perc(spec_cellex, 0.1, "ldsc/spec_cellex/41_Siletti_Cerebellum.CBV_Human_2022/genelists", "magma/spec_cellex_top10/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar")

- In addition, there are around half of the genes that are specific when using `cellex` as compared to when using `vanilla` or `ewce`. #TODO: INVESTIGATE THIS

- Save for running magma linear

In [28]:
spec_cellex_orig = pd.read_csv("scrna/41_Siletti_Cerebellum.CBV_Human_2022/41_Siletti_Cerebellum.CBV_Human_2022.esmu_fmt.csv")
# print(spec_cellex_orig.head)
spec_cellex_orig.rename(columns = {'gene':'GENE'}, inplace=True)
spec_cellex_orig = spec_cellex_orig.drop_duplicates(subset=["GENE"])
spec_cellex_orig.to_csv(os.path.join("magma/spec_cellex_linear/41_Siletti_Cerebellum.CBV_Human_2022/gene_covar/spec_cellex_linear.txt"), sep="\t", index=False) # save to a file)

<bound method NDFrame.head of                   gene  Astrocyte  Bergmann_glia  Cerebellar_inhibitory   
0      ENSG00000284678   0.000000       0.000000               0.045522  \
1      ENSG00000167995   0.278613       0.000000               0.000000   
2      ENSG00000204655   0.000000       0.000000               0.000000   
3      ENSG00000253807   0.057902       0.000000               0.000000   
4      ENSG00000169247   0.000000       0.000000               0.000000   
...                ...        ...            ...                    ...   
18908  ENSG00000167633   0.000000       0.000000               0.000000   
18909  ENSG00000221957   0.000000       0.000000               0.000000   
18910  ENSG00000005844   0.000000       0.000000               0.000000   
18911  ENSG00000122122   0.000000       0.000000               0.000000   
18912  ENSG00000246526   0.029521       0.060136               0.000000   

       Committed_oligodendrocyte_precursor  Ependymal  Fibroblast  Mi